## Physical Geography

In [12]:
%load_ext autoreload
%autoreload 2

from world_simulator import run_river_skeleton_pipeline, test_noise, generate_fractal_noise
from world_simulator import GPUThermalEroder, CoastalTaper

from world_simulator import HydrologyAnalyzer, validate_arrow_directions, plot_flow, CalculateFlowMagnitude, plot_river_hierarchy, assign_river_widths, plot_river_physics, save_hydro_network, validate_topology_continuity




import os
import geopandas as gpd
import numpy as np
import rasterio
from gdgtm import change_raster_res


vector_src_dir = "/home/pete/Documents/wfrp/source_vectors/"
raster_src_dir = "/home/pete/Documents/wfrp/source_rasters/"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Base DEM setup

Here we take a georeferenced, trimmed corase altitude map, we load it, increase resolution, and then we smooth it.

We use an erosion approach to overal smoothing and aim for Int16 data type - which gives us some serious mileage in the "disk saving department" - 100s of MBs.

The exact maths are indicated in the class docstring for the GPUThermalEroder. For now it is relevant to note that we went for a short erosion (100 steps), slow erosion rate (0.05), and a high threshold of 2. What this means is that we preserve most of the high ground on the map, while introducing just a little smoothing around the slopes to remove the blockiness. Not perfect, but the step below will take care of doing perfect :)



In [7]:
### Change resolution to 125m

src_raster = os.path.join(raster_src_dir, "wfrp_empire_topo_base_dem.tif")
dst_raster = os.path.join(raster_src_dir, "wfrp_empire_topo_base_dem_high_res.tif") 

change_raster_res(125, src_raster, dst_raster)

'Resolution meets target, file exists: /home/pete/Documents/wfrp/source_rasters/wfrp_empire_topo_base_dem_high_res.tif'

In [10]:
src_raster = os.path.join(raster_src_dir, "wfrp_empire_topo_base_dem_high_res.tif") 
dst_raster = os.path.join(raster_src_dir, "wfrp_empire_smoothed_topo.tif")

if __name__ == "__main__":
    eroder = GPUThermalEroder(talus_threshold=2, erosion_rate=0.05)
    
    # 1. Load
    raw_dem = eroder.load_dem(src_raster)
    
    # 2. Process
    smoothed_dem = eroder.process(raw_dem, iterations=100)
    
    # 3. Save
    eroder.save_dem(smoothed_dem, dst_raster)  ### Note: output is hard-coded to be Int16!
    print("Done.")


Done.


### Vectorizing the river net from QGIS polys

The goal here is to take a bunch of QGIS polygons and change them into an actual net of lines for downstream processing.

In [ ]:
# if __name__ == "__main__":
#     # Example configuration
#     IN_FILE = os.path.join(vector_src_dir, "wfrp_empire_rivers_poly.gpkg")
#     OUT_FILE = os.path.join(vector_src_dir, "wfrp_empire_rivers_line.gpkg")
    
#     # Config
#     GAP_TOLERANCE = 3500     # 20km gap filling
#     INTERP_DISTANCE = 250     # 500m vertex resolution
#     PRUNE = 3500
    
#     if os.path.exists(IN_FILE):
#         run_river_skeleton_pipeline(IN_FILE, OUT_FILE, GAP_TOLERANCE, INTERP_DISTANCE, PRUNE)
#     else:
#         print(f"File not found: {IN_FILE}")

In [ ]:
# ### Phase 1: Make the rivers flow and make them have the right size.

# if __name__ == "__main__":

#     # rivers_path = os.path.join(vector_src_dir, "wfrp_empire_rivers_line.gpkg")
#     sea_path = os.path.join(vector_src_dir, "wfrp_empire_sea_poly.gpkg")
#     lakes_path = os.path.join(vector_src_dir, "wfrp_empire_lakes_poly.gpkg")
#     rivers_path = os.path.join(vector_src_dir, "wfrp_empire_rivers_line.gpkg")
    
#     rivers = gpd.read_file(rivers_path) # The output from skeletonization
#     sea = gpd.read_file(sea_path)       # You need this
#     lakes = gpd.read_file(lakes_path)   # You need this (Canonical lakes)

#     ### Step 1: Orient the rivers in the correct direction.
#     analyzer = HydrologyAnalyzer(rivers, sea, lakes)
#     oriented_rivers = analyzer.run()
#     validate_arrow_directions(oriented_rivers, analyzer.G)
#     # plot_flow(oriented_rivers, sea, lakes)

#     ### Step 2: Establish river hierarchy
#     # 1. Get the Directed Graph from the Phase 1 Analyzer
#     flow_calculator = CalculateFlowMagnitude(analyzer.DiG)
#     # 2. Calculate Magnitude
#     dag_with_flow = flow_calculator.run()
#     # 3. Visualize
#     # plot_river_hierarchy(dag_with_flow, sea, lakes)

In [ ]:
# if __name__ == "__main__":
#     OUT_FILE = os.path.join(vector_src_dir, "wfrp_empire_major_rivers_net.gpkg")
#     ### Step 3: Work out river widths:
#     rivers_with_width = assign_river_widths(
#         dag_with_flow, 
#         min_width=20.0,    # Source streams are 20m wide
#         max_width=1000.0,   # The Reik is 800m wide at Altdorf - we make it 1000 for our 250m grid.
#         scale_factor=100.0  # Multiplier for the log growth
#     )
    
#     # 2. Restore CRS (Important for buffering!)
#     rivers_with_width.set_crs(rivers.crs, inplace=True)
    
#     # 3. Visualize
#     # plot_river_physics(rivers_with_width, sea, lakes)
#     save_hydro_network(rivers_with_width, OUT_FILE)

In [4]:
# validate_topology_continuity(rivers_with_width)

--- TOPOLOGY CONTINUITY CHECK ---
Checked 100 junctions.
SUCCESS: Water flows continuously from line to line.


### Geological "bones" and "flesh"

Here we set up the details of the hills, mountains, as well as fill in the bits below.
This is absed on expeirences of trying to use simple noise, which just failed to give us the kind of sensitivity we needed to get things done well enough.

The plan is as follows:
1. Generate tectonic ridges masked by altitude
2. Brak up the north using the North Vector
3. Apply Domain warped multifractal to erode texture terrain
4. Use coastal taper to fade things into the sea.



In [20]:
# =============================================================================
#  STEP 1: IMPORTS & ENGINE SETUP
# =============================================================================
from world_simulator.terrain_engine import CoordinateEngine
from world_simulator.layers import (
    RiverWarpLayer, 
    TectonicLayer, 
    ErosionLayer, 
    LakeIntegrationLayer,  # <--- NEW CLASS
    CoastalTaper
)


CellularReturnType.Distance2Sub


In [21]:
# =============================================================================
#  STEP 1: LOAD CONTEXT DATA
# =============================================================================

vector_src = "/home/pete/Documents/wfrp/source_vectors"
raster_src = "/home/pete/Documents/wfrp/source_rasters"

# A. The Base DEM (Canvas & Starting Heights)
base_dem_path = os.path.join(raster_src, "wfrp_empire_smoothed_topo.tif") # From your previous step
with rasterio.open(base_dem_path) as src:
    base_profile = src.profile
    base_shape = (src.height, src.width)
    base_elevation = src.read(1) # Start with existing terrain

# B. Vectors
rivers_gdf = gpd.read_file(os.path.join(vector_src, "wfrp_empire_major_rivers_net.gpkg"))
lakes_gdf = gpd.read_file(
    os.path.join(vector_src, "wfrp_empire_lakes_poly.gpkg"),
    layer="wfrp_empire_lakes")
sea_gdf = gpd.read_file(os.path.join(vector_src, "wfrp_empire_sea_poly.gpkg")) # Explicit Sea Load
north_mask_poly = gpd.read_file(os.path.join(vector_src, "wfrp_empire_north_blob.gpkg"))





In [ ]:
# =============================================================================
#  STEP 2: PRE-PROCESS COAST (Measure & Mimic)
# =============================================================================
# --- A. COASTAL ANALYSIS ---
print("Analyzing Coastline Physics...")

# 1. Rasterize (Get the shape)
raw_sea_mask = vector_to_mask(sea_gdf, base_profile)

# 2. Measure the "Native" Fractal Dimension
# We measure the complexity at coarse scales (e.g., 2km to 64km).
# This tells us if the coast is "Scottish" (High D) or "Floridian" (Low D).
# We stop at min_scale=16px (~2km) to avoid reading the square pixel artifacts.
measured_d = measure_fractal_dimension(
    mask=raw_sea_mask,
    min_scale=16,   # Ignore details smaller than ~2km (the artifacts)
    max_scale=512   # Measure up to ~64km bays
)

# Safety Clamp: Real coastlines rarely exceed 1.5 or drop below 1.1
measured_d = np.clip(measured_d, 1.1, 1.45)
print(f"  > Detected Fractal Dimension: {measured_d:.3f}")

# 3. Synthesize Detail
# We use the measured D to generate the sub-pixel details.
fractal_sea_mask = generate_fractal_mask(
    mask=raw_sea_mask,
    fractal_dimension=measured_d,  # <--- The Measurement informs the Synthesis
    scale=20.0,
    seed=123
)



In [ ]:
# =============================================================================
#  STEP 3: Initialize Engine
# =============================================================================
# We pass 'base_elevation' so we build ON TOP of the coarse terrain, not from zero.
engine = CoordinateEngine(
    shape=base_shape, 
    profile=base_profile,
    initial_elevation=base_elevation
)

In [ ]:
# =============================================================================
#  STEP 3: THE RECIPE
# =============================================================================

recipe = [
    # --- PHASE A: COORDINATE WARPING ---
    # Warps X/Y so mountains "flow" around the river valleys
    RiverWarpLayer(
        name="Major River Gravity",
        rivers_gdf=rivers_gdf,
        width_column="width",   # <--- Uses actual physical width (meters)
        warp_strength=0.0005,   # Multiplier: (Width * Strength) = Pixel Shift
        influence_km=15.0       # Max reach of the gravity well (falloff)
    ),

    # --- PHASE B: TECTONIC STRUCTURE ---
    # 1. Global Spines
    TectonicLayer(
        name="Global Spines",
        seed=42,
        frequency=0.008,
        amplitude=2500.0,
        ridge_mode="Distance2Sub",
        mask_config=None 
    ),

    # 2. Northern Fracture
    TectonicLayer(
        name="Global Spines",
        frequency=0.008,
        amplitude=1500.0,
        ridge_mode="Distance2Sub",
        mask_config={
            'type': 'altitude', 
            'min_height': 300.0,  # Starts appearing here
            'fade_height': 600.0  # Reaches full strength at 900m (300+600)
        }
    ),

    # --- PHASE C: EROSION & TEXTURE ---
    ErosionLayer(
        name="Eroded Soil",
        seed=99,
        frequency=0.05,
        amplitude=800.0,
        fractal_type="RigidMulti",
        perturb_strength=2.5,
        mask_config=None
    ),

    # --- PHASE D: HYDRO INTEGRATION (The Fix) ---
    # This runs AFTER erosion to ensure flat water and smooth banks.
    LakeIntegrationLayer(
        name="Canonical Lakes",
        lakes_gdf=lakes_gdf,
        bank_width_km=0.8,      # Blend terrain over 800m
        bed_depth_m=0.0,        # 0.0 = Flat surface. >0 would dish the bed.
        priority_mode="Auto"    # "Auto" = Detect level from shore. "Attribute" = Use column.
    ),

    # --- PHASE E: CLEANUP ---
    # 5. Final Cleanup
        SmartCoastalTaper(
            mask_array=fractal_sea_mask,  # High-res, physically accurate mask
            sea_level=0.0,
            
            # Smart Logic Config (All units in Meters)
            cliff_height_threshold=50.0,  # Elevations > 50m start becoming cliffs
            
            beach_width_m=30000.0,        # 30km fade for lowlands
            cliff_width_m=200.0,          # 200m sharp drop for highlands
            
            smoothing_sigma=2.0
        )
]

In [ ]:
# =============================================================================
#  STEP 4: EXECUTION
# =============================================================================
final_heightmap = engine.build(recipe)
print("Terrain Generation Complete.")

In [ ]:
# =============================================================================
#  STEP 5: EXPORT (The Landing)
# =============================================================================
output_path = "output/wfrp_16k_hybrid_terrain.tif"
print(f"Exporting to {output_path} (int16)...")

# Prepare Profile: Ensure LZW compression and correct Int16 type
export_profile = base_profile.copy()
export_profile.update({
    'dtype': 'int16',
    'count': 1,
    'compress': 'lzw',   # Critical for 16k rasters to save disk space
    'nodata': -32768     # Standard NoData value for Int16
})

# Save to Disk
with rasterio.open(output_path, 'w', **export_profile) as dst:
    # Round float32 to nearest int, then cast to int16
    # (Avoids truncation bias where 199.9 becomes 199)
    int_data = np.round(final_heightmap).astype('int16')
    dst.write(int_data, 1)

print("Terrain Generation Complete.")

What this will not do is getting a good coastal taper. For this we use the CoastalTaper class.
For this we set ourselves at 250 pixels from the shore (which means base DEM elevation is reached some 27.25km from the Sea of Claws). The power is the default of 2 - this means a wide coastal plain and a steep rise that still preserves the hills in the north of Nordland.

In [19]:
## Apply Coastal Taper

src_dir = "/home/pete/Documents/wfrp"
sea_vector = os.path.join(src_dir, "source_vectors/wfrp_empire_sea_poly.gpkg")
dem_raster = os.path.join(src_dir, "source_rasters/wfrp_empire_smoothed_topo.tif")
dst_raster = os.path.join(src_dir, "source_rasters/wfrp_empire_tapered_topo.tif")

# 1. Setup
# We want the land to taper off over 500 pixels from the coastline
taper_tool = CoastalTaper(taper_distance_px=250.0)

# 2. Load Inputs
# This reads the raster AND converts the vector file to a boolean mask internally
raw_dem = taper_tool.load_inputs(
    dem_path=dem_raster, 
    sea_vector=sea_vector # Can be path or GDF
)

# 3. Generate the Gradient Field
# Power 2.0 gives a parabolic curve (smooth start), 0.1 downsample makes it fast
taper_tool.generate_taper_map(power=2, downsample_factor=0.1)

# 4. Apply
# Returns float32 version with taper applied
processed_dem = taper_tool.apply(raw_dem)

# 5. Save
# Automatically converts to Int16, sets NoData = -32768, and compresses
taper_tool.save_result(processed_dem, dst_raster)

INFO: Loading inputs from /home/pete/Documents/wfrp/source_rasters/wfrp_empire_smoothed_topo.tif...
INFO: Loading Sea Vector from disk...
INFO: Rasterizing Sea Vector to match DEM profile...
INFO: Generating Coastal Taper (Dist=250.0px, Power=2)...
INFO: Applying Coastal Taper (returning float32)...
INFO: Saving result to /home/pete/Documents/wfrp/source_rasters/wfrp_empire_tapered_topo.tif as Int16...
